# Reindexer

[Reindexer](https://github.com/Restream/reindexer) - это встраиваемая, in-memory, документоориентированная база данных с интерфейсом построителя запросов высокого уровня.

Этот ноутбук показывает, как использовать векторное хранилище Reindexer с LangChain.

## Установка и настройка

```python
%pip install -qU pyreindexer langchain-reindexer langchain_huggingface sentence_transformers
```

## Базовое использование

In [1]:
from langchain_reindexer import ReindexerVectorStore
from langchain_core.embeddings import FakeEmbeddings
from langchain_core.documents import Document

Инициализация эмбеддингов (FakeEmbeddings не требует API ключей)

Для production используйте OpenAIEmbeddings, HuggingFaceEmbeddings или другие реальные эмбеддинги

In [2]:
embeddings = FakeEmbeddings(size=1024)

Создание векторного хранилища

In [3]:
vector_store = ReindexerVectorStore(
    embedding=embeddings,
    rx_connector_config={"dsn": "builtin://"},  # загрузка в память
    rx_namespace="langchain_docs",
)

### OpenAI Embeddings (требуется API ключ)

```python
from langchain_openai import OpenAIEmbeddings

# Требуется переменная окружения OPENAI_API_KEY
openai_embeddings = OpenAIEmbeddings()

openai_vector_store = ReindexerVectorStore(
    embedding=openai_embeddings,
    rx_connector_config={"dsn": "builtin:///tmp/openai_db"},
)
```

## Варианты подключения

Reindexer поддерживает различные типы подключения:

- **In-memory builtin**: База данных в памяти (по умолчанию)
  ```python
  rx_connector_config={"dsn": "builtin:///tmp/my_db"}
  ```

- **By-path builtin**: База данных по пути
  ```python
  rx_connector_config={"dsn": "builtin:///tmp/pyrx"}
  ```

- **By-auto-path builtin**: База данных с автоматическим путем на уровне запускаемого файла
  ```python
  rx_connector_config={"dsn": "builtin:///"}
  ```

Для получения дополнительной информации см. [Документация Reindexer](https://github.com/Restream/reindexer).

## Добавление документов

In [4]:
documents = [
    Document(
        page_content="LangChain - это фреймворк для разработки приложений на основе языковых моделей.",
        metadata={"topic": "framework", "year": 2023},
    ),
    Document(
        page_content="Reindexer - это встраиваемая, in-memory, документоориентированная база данных.",
        metadata={"topic": "database", "year": 2024},
    ),
    Document(
        page_content="Векторные хранилища позволяют выполнять семантический поиск по документам с использованием эмбеддингов.",
        metadata={"topic": "vector_search", "year": 2024},
    ),
]

In [5]:
# Добавление документов в векторное хранилище
ids = vector_store.add_documents(documents)
print(f"Добавлено {len(ids)} документов с ID: {ids}")

Добавлено 3 документов с ID: ['b1268f82-f6aa-4b13-8f0e-14b12cfb1888', 'c2814e38-e32d-4b58-bc63-9179d1d3501d', 'a6a94fe0-52aa-4a4d-81cc-e24b80599bb9']


## Поиск по сходству

In [6]:
# Поиск похожих документов
query = "Что такое LangChain?"
results = vector_store.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\nРезультат {i}:")
    print(f"Содержание: {doc.page_content}")
    print(f"Метаданные: {doc.metadata}")


Результат 1:
Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.
Метаданные: {'topic': 'database', 'year': 2024}

Результат 2:
Содержание: Векторные хранилища позволяют выполнять семантический поиск по документам с использованием эмбеддингов.
Метаданные: {'topic': 'vector_search', 'year': 2024}


## Поиск по сходству с оценкой

In [7]:
# Поиск с оценкой сходства
results_with_scores = vector_store.similarity_search_with_score(query, k=2)

for doc, score in results_with_scores:
    print(f"\nОценка: {score:.4f}")
    print(f"Содержание: {doc.page_content}")
    print(f"Метаданные: {doc.metadata}")


Оценка: -0.0141
Содержание: Векторные хранилища позволяют выполнять семантический поиск по документам с использованием эмбеддингов.
Метаданные: {'topic': 'vector_search', 'year': 2024}

Оценка: -0.0147
Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.
Метаданные: {'topic': 'database', 'year': 2024}


## Поиск с фильтрацией по метаданным

In [8]:
# Поиск с фильтром по метаданным
filtered_results = vector_store.similarity_search(
    query, k=2, filter={"topic": "database"}
)

for doc in filtered_results:
    print(f"Содержание: {doc.page_content}")
    print(f"Метаданные: {doc.metadata}\n")

Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.
Метаданные: {'topic': 'database', 'year': 2024}



## Поиск максимальной предельной релевантности (MMR)

In [9]:
# MMR поиск для получения разнообразных результатов
mmr_results = vector_store.max_marginal_relevance_search(
    query="фреймворк", k=2, fetch_k=3, lambda_mult=0.5
)

for doc in mmr_results:
    print(f"Содержание: {doc.page_content}")
    print(f"Метаданные: {doc.metadata}\n")

Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.
Метаданные: {'topic': 'database', 'year': 2024}

Содержание: Векторные хранилища позволяют выполнять семантический поиск по документам с использованием эмбеддингов.
Метаданные: {'topic': 'vector_search', 'year': 2024}



## Использование как Retriever

In [10]:
# Создание retriever из векторного хранилища
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Использование retriever
docs = retriever.invoke("Что такое векторное хранилище?")
for doc in docs:
    print(f"Содержание: {doc.page_content}\n")

Содержание: LangChain - это фреймворк для разработки приложений на основе языковых моделей.

Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.



## Расширенная конфигурация

In [11]:
# Создание векторного хранилища с пользовательскими параметрами HNSW
custom_vector_store = ReindexerVectorStore(
    embedding=FakeEmbeddings(size=1024),  # Используйте тот же размер эмбеддингов
    m=32,  # Больше соединений для лучшего воспроизведения
    ef_construction=400,  # Больший список кандидатов во время построения
    multithreading=1,  # Включить многопоточное построение индекса (0 отключает)
    rx_connector_config={"dsn": "builtin:///tmp/custom_db"},
    rx_namespace="custom_namespace",
)

## Сохранение и загрузка конфигурации

- `builtin://` сохраняет данные только в оперативной памяти; используйте `save_local()` для экспорта в файл `reindexer_memory_dump.json` и `load_local()` для восстановления.

- `builtin:///` автоматически создаёт директорию в каталоге запуска скрипта (`reindexer_storage/<namespace>`) и позволяет Reindexer самому управлять файлами.

- Можно указать собственный путь после `builtin:///absolute/path`, чтобы загрузить данные, подготовленные в другом окружении.

In [12]:
# Сохранение конфигурации векторного хранилища
vector_store.save_local("./reindexer_config")

In [13]:
# Загрузка конфигурации векторного хранилища
loaded_store = ReindexerVectorStore.load_local(
    "./reindexer_config", embedding=FakeEmbeddings(size=1024)
)

In [14]:
# Поиск похожих документов
query = "Что такое LangChain?"
results = loaded_store.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\nРезультат {i}:")
    print(f"Содержание: {doc.page_content}")
    print(f"Метаданные: {doc.metadata}")


Результат 1:
Содержание: Reindexer - это встраиваемая, in-memory, документоориентированная база данных.
Метаданные: {'topic': 'database', 'year': 2024}

Результат 2:
Содержание: Векторные хранилища позволяют выполнять семантический поиск по документам с использованием эмбеддингов.
Метаданные: {'topic': 'vector_search', 'year': 2024}


## Использование с реальными эмбеддингами

Для production вы можете использовать реальные модели эмбеддингов. Вот несколько примеров:

### HuggingFace Embeddings

```python
# Установите HuggingFace embeddings: %pip install langchain-huggingface sentence-transformers

from langchain_huggingface import HuggingFaceEmbeddings

# Используйте HuggingFace embeddings (работает локально, не требует API ключа)
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

hf_vector_store = ReindexerVectorStore(
    embedding=hf_embeddings,
    rx_connector_config={"dsn": "builtin:///tmp/hf_db"},
)

# Добавление документов
hf_vector_store.add_documents(documents)

# Поиск
results = hf_vector_store.similarity_search("Что такое LangChain?", k=1)
print(results[0].page_content)
```